In [1]:
import random

import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

from utils.training.data_loading import get_line_to_spectro_class
from utils.training.keras_models import resnet_model, custom_classif_model

2023-11-14 18:29:29.546119: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-14 18:29:29.785851: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 18:29:29.785913: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 18:29:29.787050: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 18:29:29.916610: I tensorflow/core/platform/cpu_feature_g

In [2]:
ROOT_DIR = "/media/plerolland/LaBoite/PublicData/training/spectrograms"
SEED = 0
BATCH_SIZE = 32
EPOCHS = 50
CHECKPOINTS_DIR = "checkpoints_classif"

SIZE = (128, 128)  # number of pixels in the spectrograms
CHANNELS = 1  # 1 means grayscale 3 RGB

data_loader = get_line_to_spectro_class(SIZE, CHANNELS)
model = custom_classif_model()

2023-11-14 18:29:33.081550: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-14 18:29:33.168049: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-14 18:29:33.168228: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
pos, neg = csv_to_lines(f"{ROOT_DIR}/dataset.csv", col_to_keep=0)
lines = pos + neg
print(len(lines), "files found")

random.Random(SEED).shuffle(lines)
train_lines = lines[int(0.2 * len(lines)):]
valid_lines = lines[:int(0.2 * len(lines))]

train_dataset = tf.data.Dataset.from_tensor_slices(train_lines)
valid_dataset = tf.data.Dataset.from_tensor_slices(valid_lines)

train_dataset = train_dataset.map(data_loader).batch(batch_size=BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE).cache()
valid_dataset = valid_dataset.map(data_loader).batch(batch_size=BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE).cache()

In [ ]:
cols = 8
lines = 4
batch_number = 1  # number of the batch we want to inspect

to_show = cols * lines
plt.figure(figsize=(20, lines*3))
shown=0
for images, y in valid_dataset.take(batch_number+to_show//BATCH_SIZE+1):
    if batch_number:
            batch_number -= 1
            continue
    for i in range(min(BATCH_SIZE, to_show-shown)):
        ax1 = plt.subplot(lines, cols, 1 + shown)
        plt.xlabel("time (s)")
        plt.ylabel("frequency (Hz)")
        plt.imshow(images[i].numpy()[:,:,0], cmap='inferno')
        label = "positive" if y[i]==1 else "negative"
        plt.title(label)

        shown += 1
plt.show()

In [ ]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss=tf.losses.binary_crossentropy,
        metrics='Accuracy')

model.build((BATCH_SIZE, SIZE[0], SIZE[1], CHANNELS))

model.summary()

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=CHECKPOINTS_DIR + "/cp-{epoch:04d}.ckpt",
                                                     save_weights_only=True,
                                                     verbose=1)

In [ ]:
model.fit(
        train_dataset,
        batch_size=BATCH_SIZE,
        validation_data=valid_dataset,
        epochs=EPOCHS,
        callbacks=[cp_callback]
    )

In [ ]:
cols = 8
lines = 4
batch_number = 1

to_show = cols * lines
plt.figure(figsize=(20, lines*3))
shown=0
for images, y in valid_dataset.take(batch_number+to_show//BATCH_SIZE+1):
    if batch_number:
            batch_number -= 1
            continue
    for i in range(min(BATCH_SIZE, to_show-shown)):
        ax1 = plt.subplot(lines, cols, 1 + shown)
        plt.xlabel("time (s)")
        plt.ylabel("frequency (Hz)")
        plt.imshow(images[i].numpy()[:,:,0], cmap='inferno')
        label = "positive" if y[i]==1 else "negative"
        predicted = model.predict(np.reshape(images[i], (1, SIZE[0], SIZE[1], CHANNELS)), verbose=False)[0]
        predicted_label = "positive" if predicted>=0.5 else "negative"
        plt.title(f"{predicted_label}/{label}")

        shown += 1
plt.show()